In [25]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

import math
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg


In [26]:
img = cv.imread('gecko_tokay.jpg', cv.IMREAD_GRAYSCALE)
h, w = img.shape

In [27]:
def create_gauss_filter(h, w, k_size=5, sigma=5):
    nucleo_g = cv.getGaussianKernel(ksize=k_size, sigma=sigma)
    kernel = nucleo_g * nucleo_g.T
    if kernel.shape[0] < h:
        zero_qty = h - kernel.shape[0]
        if zero_qty % 2 == 0:
            zero_qty = (h - kernel.shape[0])//2
            kernel = np.pad(kernel, ((zero_qty, zero_qty), (0, 0)), 'constant', constant_values=(0))
        else:
            zero_qty = (h - kernel.shape[0])//2
            kernel = np.pad(kernel, ((zero_qty, zero_qty + 1), (0, 0)), 'constant', constant_values=(0))
    if kernel.shape[1] < w:
        zero_qty = w - kernel.shape[1]
        if zero_qty % 2 == 0:
            zero_qty = (w - kernel.shape[1])//2
            kernel = np.pad(kernel, ((0, 0), (zero_qty, zero_qty)), 'constant', constant_values=(0))
        else:
            zero_qty = (w - kernel.shape[1])//2
            kernel = np.pad(kernel, ((0, 0), (zero_qty, zero_qty + 1)), 'constant', constant_values=(0))
    kernel_fft = np.fft.fft2(kernel)
    return kernel, kernel_fft

In [28]:
kernel, kernel_tf = create_gauss_filter(h, w, k_size=15, sigma=2)

plt.figure(figsize=(12,12))
plt.subplot(1,2,1)
plt.axis('off')
plt.title("Kernel Gausiano")
plt.imshow(kernel, cmap='jet')
plt.subplot(1,2,2)
plt.axis('off')
plt.title("FFT del kernel")
tf_shifted = np.fft.fftshift(kernel_tf)
plt.imshow(np.abs(tf_shifted), cmap='jet')

In [29]:
img_fft = np.fft.fft2(img)
filtered = img_fft * kernel_tf

#img_gauss = np.real(np.fft.ifft2(np.fft.ifftshift(filtered)))
img_gauss = np.fft.ifftshift(np.real(np.fft.ifft2(filtered)))

plt.figure(figsize=(12,12))
plt.subplot(1,2,1)
plt.axis('off')
plt.imshow(img, cmap='gray')
plt.title('Imagen en escala de grises')
plt.subplot(1,2,2)
plt.axis('off')
plt.imshow(img_gauss, cmap='gray')
plt.title('Desenfoque gausiano')

Text(0.5, 1.0, 'Desenfoque gausiano')

In [30]:
def ISMBIFD(img):
    img_fft = np.fft.fft2(img)
    fc = np.fft.fftshift(img_fft)
    af = np.abs(fc)
    m = np.max(af)
    threshold = m/1000
    t_h = np.count_nonzero(img_fft > threshold)
    fm = t_h/(img.shape[0]*img.shape[1])
    return fm

print(ISMBIFD(img_gauss))

0.0028138001989869753


In [31]:
# Suavizado Gaussiano usando el Kernel
#=====================================

result_b = cv.filter2D(img, -1, kernel)

In [32]:
print(ISMBIFD(result_b))

0.0028251062771345874


In [47]:
# Gray-level variance

def sta3(img):
    u = np.mean(img)
    phi = 0
    h, w = img.shape
    for i in range(h):
        for j in range(w):
            phi = phi + pow((img[i, j] - u), 2)
    return phi

In [48]:
print(sta3(img_gauss), sta3(result_b))

937725949.6664598 938011241.3345777
